In [14]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import json
import plotly.express as px
import webbrowser

from dash import Dash, dcc, html, Input, Output

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import manual_labeler_functions as man_lab_fun, general_functions as gf, files_paths as fp

## Function to analyzer the graphs of the selected video series

In [15]:
def PLOT_MEASURE_EXPLORER(df):

    port = 8051
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    app = Dash(__name__, external_stylesheets=external_stylesheets)

    def open_browser():
    	webbrowser.open_new("http://localhost:{}".format(port))
          
    app.layout = html.Div([
        html.H4('Measure Explorer'),
        html.H6('Reference Signal'),
        html.I('Setting the frame interval range'),
        html.Br(),
        dcc.Input(id='input-init', type='number', min=df.index.min(), max=df.index.max(), value=df.index.min()),
        dcc.Input(id='input-end', type='number', value=df.index.max()),
        dcc.Graph(id="graph"),
        dcc.Checklist(
            id="checklist",
            options=df.columns,
            value=["m1", "m3"],
            inline=True
        ),
        dcc.Store(
        id='data-output',
        data=[{
            'measures': ["m1", "m3"],
            'init_label': df.index.min(),
            'end_label': df.index.max()
        }]
        ),
        html.Br(),
        html.Details([
        html.Summary('Input Parameters'),
        dcc.Markdown(id='clientside-figure-json')
        ])
    ])
    
    @app.callback(
        Output("graph", "figure"), 
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_line_chart(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[frame_init:frame_end, mask]
        fig = px.line(filtered_df, width=900, height=400, labels={
                     "frame_seq": "<b>Frame number</b>",
                     "value": "<b>Amplitude (pixels)</b>",
                     "variable": "<b>Measures</b>"
                 },)
        data = measures
        fig.update_layout(
            plot_bgcolor='white',
            font_size=12,
        )
        fig.update_xaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        return fig

    @app.callback(
        Output('data-output', 'data'),
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_store_data(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[:,mask]
        return [{
            'measures': filtered_df.columns,
            'init_label': frame_init,
            'end_label': frame_end
        }]
    
    
    @app.callback(
        Output('clientside-figure-json', 'children'),
        Input('data-output', 'data')
    )
    def generated_data_json(data):
        return '```\n'+json.dumps(data, indent=2)+'\n```'
   
    if __name__ == '__main__':
        app.run_server(debug=True, port=port)

## Getting all the CSV files in the REF-Gold-Label

In [16]:
FILE_LIST_VD_MEASURE = gf.find_files_in_all_subdirectories([fp.DATASET_SEED], fp.VD_INFO)

In [17]:
FILE_LIST_VD_MEASURE

['../Dataset/REF-Gold-Label/VD_R_0000000001/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000002/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000003/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000004/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000005/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000006/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000007/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000008/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000009/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000010/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000011/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000012/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000013/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000014/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000015/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000016/VD_INFO.CSV',
 '../Dataset/REF-Gold-Label/VD_R_0000000017/VD_INFO.CSV',
 '../Dataset/R

## Generates an index with information about reference series.

In [18]:
INDEX_DT = man_lab_fun.CREATE_LABELED_INDEX(FILE_LIST_VD_MEASURE)

In [19]:
INDEX_DT

,link_video,duration_vid,total_frames,label_file_exist,path
video_id,,,,,
1,n_0,0.40,11,0,../Dataset/REF-Gold-Label/VD_R_0000000001
2,n_1,0.50,14,0,../Dataset/REF-Gold-Label/VD_R_0000000002
3,n_10,0.40,11,1,../Dataset/REF-Gold-Label/VD_R_0000000003
4,n_11,0.42,12,0,../Dataset/REF-Gold-Label/VD_R_0000000004
5,n_14,0.66,19,0,../Dataset/REF-Gold-Label/VD_R_0000000005
6,n_15,0.46,13,0,../Dataset/REF-Gold-Label/VD_R_0000000006
7,n_16,0.39,11,0,../Dataset/REF-Gold-Label/VD_R_0000000007
8,n_19,0.39,11,0,../Dataset/REF-Gold-Label/VD_R_0000000008
9,n_2,0.40,11,0,../Dataset/REF-Gold-Label/VD_R_0000000009


## Manual Process

### Selects the video for analysis and CSV data file

In [20]:
# Type the video id to label
VIDEO_ID = 3

VIDEO_FILE_PATH = str(INDEX_DT.loc[VIDEO_ID,'path'])

# Read CSV data file
VD_MEASURE_DT = man_lab_fun.READ_CSV_FILE(VIDEO_FILE_PATH, fp.VD_MEASURE_L0)

# set frames_seq as index
VD_MEASURE_DT_V2 = VD_MEASURE_DT.set_index(pd.Index(VD_MEASURE_DT['frame_seq']))
VD_MEASURE_DT_V2.drop(columns=["frame_seq", "video_id", "time_seconds"], inplace=True)

In [21]:
VD_MEASURE_DT

,video_id,frame_seq,time_seconds,m1,m3,m4,m5,m6,m7,m8,...,m14,m15,m16,m17,e1,e2,e3,b1,b2,b3
0,3,0,0.000000,8.877894,41.148035,3.845443,6.921797,12.695786,13.464542,8.877894,...,9.805879,9.805879,2.499538,6.344981,3.076354,3.100295,3.076354,11.920873,13.074506,14.233334
1,3,1,0.033367,6.986191,41.079386,3.894569,6.212484,12.406826,12.794167,10.105393,...,7.944245,9.106817,2.131383,4.456528,3.875241,3.509180,3.681479,12.788296,13.175821,15.118409
2,3,2,0.066733,7.789139,40.706639,3.894569,5.825765,12.811761,13.585471,9.719056,...,8.331769,9.106817,2.518907,4.456528,3.487717,3.487717,3.487717,12.788296,12.788296,13.956250
3,3,3,0.100100,9.127019,40.405275,3.792379,6.049474,13.212843,13.982547,8.336062,...,9.433895,9.433895,2.641491,6.037693,2.668308,3.018846,3.018846,12.075386,10.943318,13.255882
4,3,4,0.133467,5.128982,40.411252,4.246206,5.779302,11.942213,12.738619,9.679787,...,7.880886,7.112019,0.576650,4.036551,3.459901,3.099402,3.459901,13.070737,11.917437,13.860945
5,3,5,0.166833,6.955073,39.654595,4.209781,6.145628,12.202294,12.964216,10.026113,...,7.051009,10.100095,3.239653,3.239653,3.830366,3.811356,3.620789,14.483155,12.958612,16.406550
6,3,6,0.200200,1.927272,41.716434,5.794652,7.747539,12.358611,13.128107,9.827200,...,6.359999,8.672726,0.192727,0.963636,3.873770,3.469090,3.854545,14.647271,11.949089,15.461482
7,3,7,0.233567,4.183670,41.009826,5.064690,7.421212,12.526942,13.298180,9.522822,...,7.963102,8.351546,0.194222,2.913330,3.517510,3.517510,3.690218,14.760872,11.264876,15.228788
8,3,8,0.266933,2.223935,40.923885,4.972869,6.907478,11.848033,12.228668,10.638304,...,6.483829,7.246632,0.762803,1.907008,4.212719,4.195418,4.004718,14.493264,11.442050,15.679274
9,3,9,0.300300,5.142604,43.280893,4.309512,6.980878,13.540375,14.282636,10.584218,...,6.938181,9.250908,1.541818,3.083636,3.490439,3.469090,3.661818,14.647271,10.792726,14.344916


### Graph Analyse

In [22]:
PLOT_MEASURE_EXPLORER(VD_MEASURE_DT_V2)

### Loads frames from the selected video for further image analysis 


In [23]:
VIDEO_EXT = '.mp4'

# Get path from selected video id
video_name = str(INDEX_DT.loc[VIDEO_ID,'link_video']) + VIDEO_EXT

path_video_file = os.path.join(fp.VIDEO_SOURCE_SEED, video_name)

video_frames = man_lab_fun.LOAD_VIDEO_FRAMES(path_video_file)

Error opening video


### Set Frames Parameters

In [24]:
start_frame_crop = 0
end_frame_crop = len(VD_MEASURE_DT) - 1
n_frames_per_row = 6

### Displays frames for image analysis

In [25]:
man_lab_fun.DISPLAY_FRAMES(video_frames, start_frame_crop, end_frame_crop, n_frames_per_row)

<Figure size 1500x600 with 0 Axes>

### Reads or creates the VD_LABELED file if it does not exist.

In [ ]:
labeled_file_exists = int(str(INDEX_DT.loc[VIDEO_ID,'label_file_exist']))

if labeled_file_exists == 0:
    print('Creating Label File...')
    VD_MEASURE_DT['label_measures'] = str({})
    VD_LABELED_DT = VD_MEASURE_DT.copy()
else:
    print('Reading Label File...') 
    VD_LABELED_DT = man_lab_fun.READ_CSV_FILE(str(INDEX_DT.loc[VIDEO_ID,'path']).split()[2], fp.VD_LABELED_L0)

### Label the selected frames

In [ ]:
# Type the Begin and End frames you want to label (By default label the section with the selected frames above)
init_frame_label = start_frame_crop
end_frame_label = end_frame_crop

# Prepare List and Dict
selected_class = 'neutral'
label_measure_insert = ["m1", "m3", "m12", "e1", "e2"]

# Just Insert
VD_LABELED_DT = man_lab_fun.UPDATE_LABEL_DF(init_frame_label, end_frame_label, selected_class, label_measure_insert, VD_LABELED_DT)

### Verify the added labels.

In [ ]:
try:
    get_measure = man_lab_fun.GET_MEASURES_FROM_CLASS (VD_LABELED_DT, selected_class)
    print(get_measure)
except:
    print('Class not found')

### Plots a graph marking the start and end of the labels for a class.

In [ ]:
# Select the begin and end frame to be shown
start_frame = 0
end_frame = len(VD_LABELED_DT)

py.init_notebook_mode(connected=True)
plt.rcParams['font.size'] = 10
plt.rcParams["font.family"] = "Times New Roman"

man_lab_fun.PLOT_CLASS_GRAPH(VD_LABELED_DT, VD_MEASURE_DT_V2, selected_class, start_frame, end_frame)

### Save the VD_LABELED file to disk

In [ ]:
VD_LABELED_DT.to_csv(os.path.join(VIDEO_FILE_PATH, fp.VD_LABELED_L0))